実際に動いたコードしか突っ込まないこと！！！
（ここでエラーが出ると調整が大変です。)

In [27]:
import sys
# ====================================================
# Library
# ====================================================
import os
import gc
import warnings
warnings.filterwarnings('ignore')
import random
import scipy as sp
import numpy as np
import pandas as pd
from glob import glob
from pathlib import Path
import joblib
import pickle
import itertools
from tqdm.auto import tqdm
import torch
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split, GroupKFold
from sklearn.metrics import log_loss, roc_auc_score, matthews_corrcoef, f1_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import category_encoders as ce
#import lightgbm as lgb
import optuna.integration.lightgbm as lgb
import xgboost as xgb
from catboost import Pool, CatBoostRegressor, CatBoostClassifier

pd.set_option('display.max_columns',1000)
pd.set_option('display.max_rows',100)

# ====================================================
# Configurations
# ====================================================
class CFG:
    VER = 5.2
    AUTHOR = 'naokisusami'
    COMPETITION = 'FDUA2'
    DATA_PATH = Path('/data')
    OOF_DATA_PATH = Path('/oof')
    MODEL_DATA_PATH = Path('/models')
    SUB_DATA_PATH = Path('/submission')
    METHOD_LIST = ['lightgbm', 'xgboost', 'catboost']
    seed = 42
    n_folds = 7
    target_col = 'MIS_Status'
    metric = 'f1_score'
    metric_maximize_flag = True
    num_boost_round = 500
    early_stopping_round = 200
    verbose = 25
    classification_lgb_params = {
        'objective': 'binary',
        'metric': 'auc',
        'learning_rate': 0.05,
        'seed': seed,
    }
    classification_xgb_params = {
        'objective': 'binary:logistic',
        'eval_metric': 'logloss',
        'learning_rate': 0.05,
        'random_state': seed,
    }

    classification_cat_params = {
        'learning_rate': 0.05,
        'iterations': num_boost_round,
        'random_seed': seed,
    }
    model_weight_dict = {'lightgbm': 0.50, 'xgboost': 0.10, 'catboost': 0.40}
# ====================================================
# Seed everything
# ====================================================
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

seed_everything(CFG.seed)


# ====================================================
# Metric
# ====================================================
# f1_score

# ====================================================
# LightGBM Metric
# ====================================================
def lgb_metric(y_pred, y_true):
    y_true = y_true.get_label()
    return 'f1score', f1_score(y_true, np.where(y_pred >= 0.5, 1, 0), average='macro'), CFG.metric_maximize_flag

# ====================================================
# XGBoost Metric
# ====================================================
def xgb_metric(y_pred, y_true):
    y_true = y_true.get_label()
    return 'f1score', f1_score(y_true, np.where(y_pred >= 0.5, 1, 0), average='macro')


In [6]:
#LGB用データ作成
#データの読み込み
train_df = pd.read_csv('train.csv', index_col=0)
test_df = pd.read_csv('test.csv', index_col=0)
categorical_features = ['RevLineCr', 'LowDoc', 'UrbanRural', 'State', 'Sector']

#前処理メソッドの定義
def Preprocessing(input_df: pd.DataFrame()) -> pd.DataFrame():
    #欠損値に対する処理
    def deal_missing(input_df: pd.DataFrame()) -> pd.DataFrame():
        df = input_df.copy()
        for col in ['RevLineCr', 'LowDoc', 'BankState']:
            df[col] = input_df[col].fillna('UNK')
        for col in ['DisbursementDate','ApprovalDate']:
            df[col] = input_df[col].fillna('50-NaN-50')
        
        return df
    #金額に対する前処理
    def clean_money(input_df: pd.DataFrame()) -> pd.DataFrame():
        df = input_df.copy()
        for col in ['DisbursementGross', 'GrAppv', 'SBA_Appv']:
            df[col] = input_df[col].str[1:].str.replace(',', '').str.replace(' ', '').astype(float)
        return df
    df = deal_missing(input_df)
    df = clean_money(df)
    df['NewExist'] = np.where(input_df['NewExist'] == 1, 1, 0)
    #特徴量作成
    def make_features(input_df: pd.DataFrame()) -> pd.DataFrame():
        df = input_df.copy()
        #日付関係の特徴量作成
        df[['DisbursementDay','DisbursementMonth','DisbursementYear']] = df['DisbursementDate'].str.split('-',expand=True)
        df[['ApprovalDay','ApprovalMonth','ApprovalYear']] = df['ApprovalDate'].str.split('-',expand=True)
        df['DisbursementDay'] = df['DisbursementDay'].astype(int)
        df['DisbursementYear'] = df['DisbursementYear'].astype(int)
        df['ApprovalDay'] = df['ApprovalDay'].astype(int)
        df['ApprovalYear'] = df['ApprovalYear'].astype(int)
        Month_dict = {'Jan':1,'Feb':2,'Mar':3,'Apr':4,'May':5,'Jun':6,'Jul':7,'Aug':8,'Sep':9,'Oct':10,'Nov':11,'Dec':12,'NaN':50}
        df['DisbursementMonth'] = df['DisbursementMonth'].map(Month_dict)
        df['ApprovalMonth'] = df['ApprovalMonth'].map(Month_dict)
        df['DisbursementDate'] = df['DisbursementYear'].astype(str)+df['DisbursementMonth'].astype(str)+df['DisbursementDay'].astype(str)
        df['DisbursementYear'] = df['DisbursementYear'].apply(lambda x:x - 100 if x >50 else x)
        df['ApprovalYear'] = df['ApprovalYear'].apply(lambda x:x - 100 if x >50 else x)
        df['CompanyLong'] = df['DisbursementYear'] - df['ApprovalYear']
        df['ApprovalTerm'] = 15 - df['ApprovalFY']
        df['DisbursementTerm'] = 15 - df['DisbursementYear']

        #Bankraptcydataの74~80は生成したもので実際の数値ではない。(失業率から換算して生成)
        Bankraptcydata={-26:32700,-25:52200,-24:46200,-23:42300,-22:36300,-21:34200,-20:46200,-19:44000,-18:48500,-17:69800,-16:62500,
                      -15:64500,-14:72000,-13:81500,-12:83000,-11:64500,-10:65000,-9:67000,-8:71000,-7:67000,-6:58000,-5:51000,
                        -4:52500,-3:54000,-2:51000,-1:41000,0:37500,1:35992,2:39845,3:37548,4:36785,5:31952,6:35292,7:21960,8:30741,
                        9:49091,10:61148,11:54212,12:46393,13:37552,14:31671,15:26130,16:24797,17:23591,18:23106,19:22157}

        #年ごとのデータを、1-5年後の平均に変換
        datalist = [Bankraptcydata]
        for k in datalist:
            for i in range(len(k)-5):
                k[-27+i] = 0
                for j in range(5):
                    k[-27+i] += k[-26+i+j]
                k[-27+i] = k[-27+i]/5
            k[50] = k[-26]*2
        
        df['Bankraptcy_By_Year'] = df['DisbursementYear'].map(Bankraptcydata)
        
        #State関係の特徴量作成
        df['Cor_State'] = (df['State']==df['BankState']).astype(int)
        StateList = ['AL','AK','AZ','AR','CA','CO','CT','DE','DC','FL','GA','HI','ID','IL','IN','IA','KS','KY','LA','ME','MD','MA',
                      'MI','MN','MS','MO','MT','NE','NV','NH','NJ','NM','NY','NC','ND','OH','OK','OR','PA','RI','SC','SD','TN','TX',
                      'UT','VT','VA','WA','WV','WI','WY']
        
        UnemploymentList = [2.6,3.7,4.0,3.4,4.1,2.8,4.0,4.6,4.2,2.7,3.1,3.7,2.8,4.6,3.1,3.0,2.9,3.9,3.5,3.1,3.0,3.7,4.3,2.9,4.0,
                          2.7,2.6,2.7,5.5,2.9,3.3,3.5,4.1,3.8,2.1,4.1,3.2,4.8,4.3,3.2,3.3,2.2,3.5,3.8,2.4,3.0,3.1
                            ,4.5,4.1,3.0,3.9]

        GDPList = [29603,44807,33655,27781,42376,40805,51911,56496,126421,33417,35265,38850,29843,39568,32724,35814,34770,30364,35181,
                   30282,39596,47351,32846,41353,24477,32590,28201,37075,40210,37375,45052,30943,49038,37053,34694,34040,29470,
                   38339,35153,36543,28894,35596,33742,37793,32774,34197,41617,40361,24929,34890,40303]
        
        GDPperPersonList = [37282,71008,48148,35674,53525,54943,63504,76720,164002,45958,48434,50788,39529,49083,40529,44091,43633,
                       38148,48366,37734,50729,55364,38433,51829,31127,41012,37966,46803,63662,46400,55320,41878,58126,49625,43172,
                       41073,40376,46248,43246,44738,38093,44955,42865,54766,47313,40312,54102,52810,31914,43309,63822]
        
        AveSalaryList = [40.46,50.81,45.40,37.79,56.10,49.79,60.14,49.66,79.85,43.66,46.17,44.09,36.45,51.71,40.97,38.39,40.96,
                        39.54,43.15,39.06,54.28,58.62,45.19,46.99,35.95,42.58,35.81,39.87,44.38,46.38,56.72,40.91,61.04,43.11,41.12,
                        43.45,40.75,43.46,46.10,46.38,39.63,35.00,41.88,48.35,41.11,39.54,52.07,51.04,38.48,41.46,44.03]
        
        Unemploymentdict = dict(zip(StateList,UnemploymentList))
        GDPdict = dict(zip(StateList,GDPList))
        GDPperPersondict = dict(zip(StateList,GDPperPersonList))
        AveSalarydict = dict(zip(StateList,AveSalaryList))
        
        df['Unemployment_By_State'] = df['State'].map(Unemploymentdict)
        df['GDP_By_State'] = df['State'].map(GDPdict)
        df['GDPperPerson_By_State'] = df['State'].map(GDPperPersondict)
        df['AveSalary_By_State'] = df['State'].map(AveSalarydict)
        
        
        #現状グループ分けされない特徴量の作成
        #企業の安定さ、デカさ
        df['BCI'] = df['CompanyLong']*(df['NoEmp'])*(df['NewExist']+1)
        df['BCI'] = df['BCI'].fillna(df['BCI'].mean)
        
        #しんどさ指数2
        df['TI2'] = (df['SBA_Appv']/(df['NoEmp']+df['CreateJob']+1))/(df['Term']+1)
        
        #派生特徴量
        # Employee to Loan Size Ratio
        # 財務関連の派生特徴量
        df['GrAppv_SBA_Appv_ratio'] = df['SBA_Appv'] / (df['GrAppv'] + 1) 
        df['DisbursementGross_GrAppv_ratio'] = df['DisbursementGross'] / (df['GrAppv'] + 1)
        # ビジネスの条件関連
        df['NewExist_NoEmp_interaction'] = df['NewExist'] * df['NoEmp']
        df['UrbanRural_Sector_interaction'] = df['UrbanRural'].astype(str) + '_' + df['Sector'].astype(str)
        # リスク関連の派生特徴量
        df['RevLineCr_LowDoc_risk_indicator'] = (df['RevLineCr'] == 'Y').astype(int) + (df['LowDoc'] == 'Y').astype(int)
        # FranchiseCodeのリスク要因
        df['Franchise_risk_factor'] = df['FranchiseCode'].apply(lambda x: 0 if x in [0, 1] else 1)

        #組み合わせ特徴量
        df['State_Sector'] = df['State'].astype(str) + '_' + df['Sector'].astype(str)
        # 地理的特徴の組み合わせ
        df['City_State'] = df['City'] + '_' + df['State']
        df['Lender_Borrower_SameState'] = (df['BankState'] == df['State']).astype(int)
        # 経済的特徴の組み合わせ
        df['Emp_to_Loan_Ratio'] = df['NoEmp'] / (df['DisbursementGross'] + 1)
        df['JobImpactScore'] = df['CreateJob'] + df['RetainedJob']
        df['Employment_creation_ratio'] = df['CreateJob'] / (df['NoEmp'] + 1)
        df['Disbursement_per_Term'] = df['DisbursementGross'] / (df['Term']+1)
        # 業種と金融条件の組み合わせ
        df['Sector_RevLineCr'] = df['Sector'].astype(str) + '_' + df['RevLineCr']
        df['Sector_LowDoc'] = df['Sector'].astype(str) + '_' + df['LowDoc']
        # 時間的特徴の組み合わせ
        df['ApprovalFY_Term'] = df['ApprovalFY'].astype(str) + '_' + df['Term'].astype(str)


        


        
        return df
    df = make_features(df)
    return df

#前処理の実行
train_df = Preprocessing(train_df)
test_df = Preprocessing(test_df)

#カウントエンコーディング
for col in ['FranchiseCode','UrbanRural', 'State','City', 'Sector', 'City_State',
            'ApprovalFY_Term','UrbanRural_Sector_interaction', 'Sector_RevLineCr', 'Sector_LowDoc','State_Sector']:
    count_dict = dict(train_df[col].value_counts())
    train_df[f'{col}_count_encoding'] = train_df[col].map(count_dict).astype(int)
    test_df[f'{col}_count_encoding'] = test_df[col].map(count_dict).fillna(1).astype(int)

#ラベルエンコーディング
for col in categorical_features :
    le = LabelEncoder()
    le.fit(train_df[col])
    train_df[col] = le.transform(train_df[col])
    test_df[col] = le.transform(test_df[col])
    
#OneHotEncoding
train_df2 = train_df.drop(['MIS_Status'],axis=1)
OneHotList = ['RevLineCr', 'LowDoc']
ohe = ce.OneHotEncoder(cols=OneHotList,use_cat_names=True)
train_df2 = ohe.fit_transform(train_df2)
test_df = ohe.transform(test_df)
train_df = pd.concat([train_df2,train_df['MIS_Status']],axis=1)

#featuresの作成
categorical_features =['UrbanRural', 'State', 'Sector','RevLineCr_0.0','RevLineCr_1.0','RevLineCr_2.0','RevLineCr_3.0','RevLineCr_4.0',
                       'LowDoc_0.0','LowDoc_1.0','LowDoc_2.0','LowDoc_3.0','LowDoc_4.0','LowDoc_5.0','LowDoc_6.0','FranchiseCode']

RemoveList=['MIS_Status','City','BankState','ApprovalDate','DisbursementDate','ApprovalFY_Term','UrbanRural_Sector_interaction',
            'Sector_RevLineCr', 'Sector_LowDoc','State_Sector','City_State']
features = train_df.columns.tolist()
for i in RemoveList:
    print(i)
    features.remove(i)
#専用変数として保存
train_df_lgb = train_df
test_df_lgb = test_df
categorical_features_lgb = categorical_features
features_lgb = features

MIS_Status
City
BankState
ApprovalDate
DisbursementDate
ApprovalFY_Term
UrbanRural_Sector_interaction
Sector_RevLineCr
Sector_LowDoc
State_Sector
City_State


In [7]:
#CTB用データ作成
#データの読み込み
train_df = pd.read_csv('train.csv', index_col=0)
test_df = pd.read_csv('test.csv', index_col=0)
categorical_features = ['RevLineCr', 'LowDoc', 'UrbanRural', 'State', 'Sector']
#前処理メソッドの定義
def Preprocessing(input_df: pd.DataFrame()) -> pd.DataFrame():
    #欠損値に対する処理
    def deal_missing(input_df: pd.DataFrame()) -> pd.DataFrame():
        df = input_df.copy()
        for col in ['RevLineCr', 'LowDoc', 'BankState']:
            df[col] = input_df[col].fillna('UNK')
        for col in ['DisbursementDate','ApprovalDate']:
            df[col] = input_df[col].fillna('50-NaN-50')
        
        return df
    #金額に対する前処理
    def clean_money(input_df: pd.DataFrame()) -> pd.DataFrame():
        df = input_df.copy()
        for col in ['DisbursementGross', 'GrAppv', 'SBA_Appv']:
            df[col] = input_df[col].str[1:].str.replace(',', '').str.replace(' ', '').astype(float)
        return df
    df = deal_missing(input_df)
    df = clean_money(df)
    df['NewExist'] = np.where(input_df['NewExist'] == 1, 1, 0)
    #特徴量作成
    def make_features(input_df: pd.DataFrame()) -> pd.DataFrame():
        df = input_df.copy()
        #日付関係の特徴量作成
        df[['DisbursementDay','DisbursementMonth','DisbursementYear']] = df['DisbursementDate'].str.split('-',expand=True)
        df[['ApprovalDay','ApprovalMonth','ApprovalYear']] = df['ApprovalDate'].str.split('-',expand=True)
        df['DisbursementDay'] = df['DisbursementDay'].astype(int)
        df['DisbursementYear'] = df['DisbursementYear'].astype(int)
        df['ApprovalDay'] = df['ApprovalDay'].astype(int)
        df['ApprovalYear'] = df['ApprovalYear'].astype(int)
        Month_dict = {'Jan':1,'Feb':2,'Mar':3,'Apr':4,'May':5,'Jun':6,'Jul':7,'Aug':8,'Sep':9,'Oct':10,'Nov':11,'Dec':12,'NaN':50}
        df['DisbursementMonth'] = df['DisbursementMonth'].map(Month_dict)
        df['ApprovalMonth'] = df['ApprovalMonth'].map(Month_dict)
        df['DisbursementDate'] = df['DisbursementYear'].astype(str)+df['DisbursementMonth'].astype(str)+df['DisbursementDay'].astype(str)
        df['DisbursementYear'] = df['DisbursementYear'].apply(lambda x:x - 100 if x >50 else x)
        df['ApprovalYear'] = df['ApprovalYear'].apply(lambda x:x - 100 if x >50 else x)
        df['CompanyLong'] = df['DisbursementYear'] - df['ApprovalYear']

        #Bankraptcydataの74~80は生成したもので実際の数値ではない。(失業率から換算して生成)
        Bankraptcydata={-26:32700,-25:52200,-24:46200,-23:42300,-22:36300,-21:34200,-20:46200,-19:44000,-18:48500,-17:69800,-16:62500,
                      -15:64500,-14:72000,-13:81500,-12:83000,-11:64500,-10:65000,-9:67000,-8:71000,-7:67000,-6:58000,-5:51000,
                        -4:52500,-3:54000,-2:51000,-1:41000,0:37500,1:35992,2:39845,3:37548,4:36785,5:31952,6:35292,7:21960,8:30741,
                        9:49091,10:61148,11:54212,12:46393,13:37552,14:31671,15:26130,16:24797,17:23591,18:23106,19:22157}

        #年ごとのデータを、1-5年後の平均に変換
        datalist = [Bankraptcydata]
        for k in datalist:
            for i in range(len(k)-5):
                k[-27+i] = 0
                for j in range(5):
                    k[-27+i] += k[-26+i+j]
                k[-27+i] = k[-27+i]/5
            k[50] = k[-26]*2
        
        df['Bankraptcy_By_Year'] = df['DisbursementYear'].map(Bankraptcydata)

        
         # 地理的特徴の組み合わせ
        df['City_State'] = df['City'] + '_' + df['State']
        # 時間的特徴の組み合わせ
        df['ApprovalFY_Term'] = df['ApprovalFY'].astype(str) + '_' + df['Term'].astype(str)
        


        
        return df
    df = make_features(df)
    return df

#前処理の実行
train_df = Preprocessing(train_df)
test_df = Preprocessing(test_df)

#ラベルエンコーディング
for col in categorical_features :
    le = LabelEncoder()
    le.fit(train_df[col])
    train_df[col] = le.transform(train_df[col])
    test_df[col] = le.transform(test_df[col])
    
categorical_features_unlabelable = ['ApprovalFY_Term','City_State','City','ApprovalDate','BankState','DisbursementDate']
'''
for col in categorical_features_unlabelable:
    le = LabelEncoder()   
    le.fit(train_df[col])
    train_df[col] = le.transform(train_df[col])
    test_df[col] = test_df[col].apply(lambda x: le.transform([x])[0] if x in le.classes_ else len(le.classes_))
'''
for col in categorical_features_unlabelable:
    encoder = LabelEncoder()
    combined = pd.concat([train_df[col], test_df[col]], axis=0)
    encoder.fit(combined)
    train_df[col] = encoder.transform(train_df[col])
    test_df[col] = encoder.transform(test_df[col])
#OneHotEncoding
train_df2 = train_df.drop(['MIS_Status'],axis=1)
OneHotList = ['RevLineCr', 'LowDoc']
ohe = ce.OneHotEncoder(cols=OneHotList,use_cat_names=True)
train_df2 = ohe.fit_transform(train_df2)
test_df = ohe.transform(test_df)
train_df = pd.concat([train_df2,train_df['MIS_Status']],axis=1)

#featuresの作成
categorical_features =['UrbanRural', 'State', 'Sector','RevLineCr_0.0','RevLineCr_1.0','RevLineCr_2.0','RevLineCr_3.0','RevLineCr_4.0',
                       'LowDoc_0.0','LowDoc_1.0','LowDoc_2.0','LowDoc_3.0','LowDoc_4.0','LowDoc_5.0','LowDoc_6.0','FranchiseCode',
                       'ApprovalFY_Term','City_State','City','ApprovalDate','BankState']
RemoveList=['MIS_Status']
features = train_df.columns.tolist()
for i in RemoveList:
    print(i)
    features.remove(i)
    
#専用変数の作成
train_df_ctb = train_df
test_df_ctb = test_df
categorical_features_ctb = categorical_features
features_ctb = features

MIS_Status


In [8]:
#lightgbmでの学習メソッドの定義
def lightgbm_training(x_train: pd.DataFrame, y_train: pd.DataFrame, x_valid: pd.DataFrame, y_valid: pd.DataFrame, features: list, categorical_features: list):
    lgb_train = lgb.Dataset(x_train, y_train, categorical_feature=categorical_features)
    lgb_valid = lgb.Dataset(x_valid, y_valid, categorical_feature=categorical_features)
    model = lgb.train(
                params = CFG.classification_lgb_params,
                train_set = lgb_train,
                num_boost_round = CFG.num_boost_round,
                valid_sets = [lgb_train, lgb_valid],
                feval = lgb_metric,
                callbacks=[lgb.early_stopping(stopping_rounds=CFG.early_stopping_round,
                                              verbose=CFG.verbose)]
            )
    # Predict validation
    valid_pred = model.predict(x_valid)
    return model, valid_pred

#xgboostでの学習メソッドの定義
def xgboost_training(x_train: pd.DataFrame, y_train: pd.DataFrame, x_valid: pd.DataFrame, y_valid: pd.DataFrame, features: list, categorical_features: list):
    xgb_train = xgb.DMatrix(data=x_train, label=y_train)
    xgb_valid = xgb.DMatrix(data=x_valid, label=y_valid)
    model = xgb.train(
                CFG.classification_xgb_params,
                dtrain = xgb_train,
                num_boost_round = CFG.num_boost_round,
                evals = [(xgb_train, 'train'), (xgb_valid, 'eval')],
                early_stopping_rounds = CFG.early_stopping_round,
                verbose_eval = CFG.verbose,
                feval = xgb_metric,
                maximize = CFG.metric_maximize_flag,
            )
    # Predict validation
    valid_pred = model.predict(xgb.DMatrix(x_valid))
    return model, valid_pred

#catboostでの学習メソッドの定義
def catboost_training(x_train: pd.DataFrame, y_train: pd.DataFrame, x_valid: pd.DataFrame, y_valid: pd.DataFrame, features: list, categorical_features: list):
    cat_train = Pool(data=x_train, label=y_train, cat_features=categorical_features)
    cat_valid = Pool(data=x_valid, label=y_valid, cat_features=categorical_features)
    model = CatBoostClassifier(**CFG.classification_cat_params)
    model.fit(cat_train,
              eval_set = [cat_valid],
              early_stopping_rounds = CFG.early_stopping_round,
              verbose = CFG.verbose,
              use_best_model = True)
    # Predict validation
    valid_pred = model.predict_proba(x_valid)[:, 1]
    return model, valid_pred
#任意のモデルでのクロスバリデーション学習メソッドの定義
def gradient_boosting_model_cv_training(method: str, train_df: pd.DataFrame, features: list, categorical_features: list):
    # Create a numpy array to store out of folds predictions
    oof_predictions = np.zeros(len(train_df))
    oof_fold = np.zeros(len(train_df))
    kfold = KFold(n_splits=CFG.n_folds, shuffle=True, random_state=CFG.seed)
    for fold, (train_index, valid_index) in enumerate(kfold.split(train_df)):
        print('-'*50)
        print(f'{method} training fold {fold+1}')

        x_train = train_df[features].iloc[train_index]
        y_train = train_df[CFG.target_col].iloc[train_index]
        x_valid = train_df[features].iloc[valid_index]
        y_valid = train_df[CFG.target_col].iloc[valid_index]
        if method == 'lightgbm':
            model, valid_pred = lightgbm_training(x_train, y_train, x_valid, y_valid, features, categorical_features)
        if method == 'xgboost':
            model, valid_pred = xgboost_training(x_train, y_train, x_valid, y_valid, features, categorical_features)
        if method == 'catboost':
            model, valid_pred = catboost_training(x_train, y_train, x_valid, y_valid, features, categorical_features)

        # Save best model
        pickle.dump(model, open(f'{method}_fold{fold + 1}_seed{CFG.seed}_ver{CFG.VER}.pkl', 'wb'))
        # Add to out of folds array
        oof_predictions[valid_index] = valid_pred
        oof_fold[valid_index] = fold + 1
        del x_train, x_valid, y_train, y_valid, model, valid_pred
        gc.collect()

    # Compute out of folds metric
    score = f1_score(train_df[CFG.target_col], oof_predictions >= 0.5, average='macro')
    print(f'{method} our out of folds CV f1score is {score}')
    # Create a dataframe to store out of folds predictions
    oof_df = pd.DataFrame({CFG.target_col: train_df[CFG.target_col], f'{method}_prediction': oof_predictions, 'fold': oof_fold})
    oof_df.to_csv(f'oof_{method}_seed{CFG.seed}_ver{CFG.VER}.csv', index = False)
#学習メソッドの定義
def Learning():
    gradient_boosting_model_cv_training('lightgbm', train_df_lgb, features_lgb, categorical_features_lgb)
    gradient_boosting_model_cv_training('xgboost',train_df_ctb, features_ctb, categorical_features_ctb)
    gradient_boosting_model_cv_training('catboost',train_df_ctb, features_ctb, categorical_features_ctb)

In [9]:
Learning()

[I 2024-02-08 15:50:33,590] A new study created in memory with name: no-name-15eef688-9525-4522-b74d-fdd7b088b265


--------------------------------------------------
lightgbm training fold 1


feature_fraction, val_score: -inf:   0%|                                                         | 0/7 [00:00<?, ?it/s]

[LightGBM] [Warning] Met categorical feature which contains sparse values. Consider renumbering to consecutive integers started from zero
[LightGBM] [Info] Number of positive: 32336, number of negative: 3927
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.026104 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 5149
[LightGBM] [Info] Number of data points in the train set: 36263, number of used features: 63
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.891708 -> initscore=2.108305
[LightGBM] [Info] Start training from score 2.108305
Training until validation scores don't improve for 200 rounds


feature_fraction, val_score: 0.756429:  14%|######4                                      | 1/7 [00:36<03:36, 36.08s/it]

...

Early stopping, best iteration is:
[92]	valid_0's auc: 0.934727	valid_0's f1score: 0.698148	valid_1's auc: 0.775567	valid_1's f1score: 0.643856


lightgbm our out of folds CV f1score is 0.6371000033921015
--------------------------------------------------
xgboost training fold 1
[0]	train-logloss:0.65990	train-f1score:0.09771	eval-logloss:0.65991	eval-f1score:0.09208
[25]	train-logloss:0.33453	train-f1score:0.67453	eval-logloss:0.33704	eval-f1score:0.64129
[50]	train-logloss:0.27755	train-f1score:0.67064	eval-logloss:0.28654	eval-f1score:0.63481
[75]	train-logloss:0.25977	train-f1score:0.68020	eval-logloss:0.27734	eval-f1score:0.64236
[100]	train-logloss:0.24999	train-f1score:0.68765	eval-logloss:0.27553	eval-f1score:0.64371
[125]	train-logloss:0.24230	train-f1score:0.69477	eval-logloss:0.27526	eval-f1score:0.64601
[150]	train-logloss:0.23701	train-f1score:0.70069	eval-logloss:0.27540	eval-f1score:0.64924
[175]	train-logloss:0.23231	train-f1score:0.70406	eval-logloss:0.27566	eval-f1score:0.64752
[200]	train-logloss:0.22736	train-f1score:0.71113	eval-logloss:0.27597	eval-f1score:0.65264
[225]	train-logloss:0.22372	train-f1score:0

[450]	train-logloss:0.19528	train-f1score:0.76863	eval-logloss:0.27544	eval-f1score:0.67500
[475]	train-logloss:0.19261	train-f1score:0.77498	eval-logloss:0.27590	eval-f1score:0.67189
[499]	train-logloss:0.18841	train-f1score:0.78268	eval-logloss:0.27615	eval-f1score:0.67006
--------------------------------------------------
xgboost training fold 6
[0]	train-logloss:0.65976	train-f1score:0.09575	eval-logloss:0.66044	eval-f1score:0.10380
[25]	train-logloss:0.33313	train-f1score:0.66029	eval-logloss:0.34500	eval-f1score:0.66320
[50]	train-logloss:0.27622	train-f1score:0.66274	eval-logloss:0.29736	eval-f1score:0.66046
[75]	train-logloss:0.25812	train-f1score:0.67489	eval-logloss:0.28904	eval-f1score:0.66995
[100]	train-logloss:0.24831	train-f1score:0.68264	eval-logloss:0.28730	eval-f1score:0.67672
[125]	train-logloss:0.24207	train-f1score:0.68900	eval-logloss:0.28721	eval-f1score:0.67555
[150]	train-logloss:0.23672	train-f1score:0.69489	eval-logloss:0.28713	eval-f1score:0.67780
[175]	trai

325:	learn: 0.2575422	test: 0.2753398	best: 0.2753183 (315)	total: 30.6s	remaining: 16.3s
350:	learn: 0.2561949	test: 0.2752824	best: 0.2752623 (346)	total: 33s	remaining: 14s
375:	learn: 0.2546850	test: 0.2751707	best: 0.2751043 (372)	total: 35.4s	remaining: 11.7s
400:	learn: 0.2532164	test: 0.2751125	best: 0.2751034 (396)	total: 37.9s	remaining: 9.35s
425:	learn: 0.2516511	test: 0.2749291	best: 0.2749291 (425)	total: 40.3s	remaining: 7s
450:	learn: 0.2505892	test: 0.2747473	best: 0.2747324 (444)	total: 42.7s	remaining: 4.64s
475:	learn: 0.2494744	test: 0.2747472	best: 0.2746401 (455)	total: 45.2s	remaining: 2.28s
499:	learn: 0.2483044	test: 0.2749072	best: 0.2746401 (455)	total: 47.5s	remaining: 0us

bestTest = 0.2746401408
bestIteration = 455

Shrink model to first 456 iterations.
--------------------------------------------------
catboost training fold 4
0:	learn: 0.6453655	test: 0.6446511	best: 0.6446511 (0)	total: 91.6ms	remaining: 45.7s
25:	learn: 0.3095927	test: 0.3033003	best:

400:	learn: 0.2545533	test: 0.2767349	best: 0.2764830 (233)	total: 40.4s	remaining: 9.97s
425:	learn: 0.2532389	test: 0.2767797	best: 0.2764830 (233)	total: 43s	remaining: 7.47s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.2764830279
bestIteration = 233

Shrink model to first 234 iterations.
catboost our out of folds CV f1score is 0.6440738667508445


In [10]:
def lightgbm_inference(x_test: pd.DataFrame):
    test_pred = np.zeros(len(x_test))
    for fold in range(CFG.n_folds):
        model = pickle.load(open(f'lightgbm_fold{fold + 1}_seed{CFG.seed}_ver{CFG.VER}.pkl', 'rb'))
        # Predict
        pred = model.predict(x_test)
        test_pred += pred
    return test_pred / CFG.n_folds
def xgboost_inference(x_test: pd.DataFrame):
    test_pred = np.zeros(len(x_test))
    for fold in range(CFG.n_folds):
        model = pickle.load(open(f'xgboost_fold{fold + 1}_seed{CFG.seed}_ver{CFG.VER}.pkl', 'rb'))
        # Predict
        pred = model.predict(xgb.DMatrix(x_test))
        test_pred += pred
    return test_pred / CFG.n_folds

def catboost_inference(x_test: pd.DataFrame):
    test_pred = np.zeros(len(x_test))
    for fold in range(CFG.n_folds):
        model = pickle.load(open(f'catboost_fold{fold + 1}_seed{CFG.seed}_ver{CFG.VER}.pkl', 'rb'))
        # Predict
        pred = model.predict_proba(x_test)[:, 1]
        test_pred += pred
    return test_pred / CFG.n_folds

def gradient_boosting_model_inference(method: str):
    x_test_lgb = test_df_lgb[features_lgb]
    x_test_ctb = test_df_ctb[features_ctb]
    if method == 'lightgbm':
        test_pred = lightgbm_inference(x_test_lgb)
    if method == 'xgboost':
        test_pred = xgboost_inference(x_test_ctb)
    if method == 'catboost':
        test_pred = catboost_inference(x_test_ctb)
    return test_pred

def Predicting():
    output_df = test_df_lgb.copy()
    output_df['pred_prob'] = 0
    for method in CFG.METHOD_LIST:
        output_df[f'{method}_pred_prob'] = gradient_boosting_model_inference(method)
        output_df['pred_prob'] += CFG.model_weight_dict[method] * output_df[f'{method}_pred_prob']
    return output_df

In [11]:
test_df_lgb = Predicting()

In [12]:
#後処理の定義
def Postprocessing():
    train_df_lgb['pred_prob'] = 0
    for method in CFG.METHOD_LIST:
        oof_df = pd.read_csv(f'oof_{method}_seed{CFG.seed}_ver{CFG.VER}.csv')
        train_df_lgb['pred_prob'] += CFG.model_weight_dict[method] * oof_df[f'{method}_prediction']
    best_score = 0
    best_v = 0
    for v in tqdm(np.arange(1000) / 1000):
        score = f1_score(oof_df[CFG.target_col], train_df_lgb[f'pred_prob'] >= v, average='macro')
        if score > best_score:
            best_score = score
            best_v = v
    print(best_score, best_v)
    test_df_lgb['target'] = np.where(test_df_lgb['pred_prob'] >= best_v, 1, 0)
    return train_df_lgb, test_df_lgb

In [28]:
#後処理
train_df, test_df = Postprocessing()

  0%|          | 0/1000 [00:00<?, ?it/s]

0.686757845822522 0.758


In [14]:
test_df[['target']].to_csv(f'seed{CFG.seed}_ver{CFG.VER}_{CFG.AUTHOR}_submission.csv', header=False)